In [ ]:
# LOADING THE DATA
import pickle

# Load training data
training_data = []
with open("/content/drive/MyDrive/chess_training_data/training_data2.pkl", "rb") as f:
    while True:
        try:
            training_data.append(pickle.load(f))
        except EOFError:
            break

# Loading labels data
labels_data = []
with open("/content/drive/MyDrive/chess_training_data/labels2.pkl", "rb") as f:
    while True:
        try:
            labels_data.append(pickle.load(f))
        except EOFError:
            break


# Check the lengths of the loaded data
print(len(training_data))
print(len(labels_data))

11187
11187


In [ ]:
import numpy as np

# Getting data values from list of np arrays
labels_data = np.array([data for data in labels_data])
print(labels_data.shape)

# Given the training data a dimension of 1
training_data = np.expand_dims(training_data, axis=-1)
print(training_data.shape)

(11187, 4)
(11187, 8, 8, 1)


In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical

In [ ]:
# Define the CNN architecture
input_layer = layers.Input(shape=(8, 8, 1))

# Shared layers
x = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)

# Separate output layers for each coordinate (classification)
output1 = layers.Dense(8, activation='softmax', name='output1')(x)  # Coordinate 1
output2 = layers.Dense(8, activation='softmax', name='output2')(x)  # Coordinate 2
output3 = layers.Dense(8, activation='softmax', name='output3')(x)  # Coordinate 3
output4 = layers.Dense(8, activation='softmax', name='output4')(x)  # Coordinate 4

# Create the model
model = models.Model(inputs=input_layer, outputs=[output1, output2, output3, output4])

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 8, 8, 1)]            0         []                            
                                                                                                  
 conv2d_12 (Conv2D)          (None, 6, 6, 32)             320       ['input_4[0][0]']             
                                                                                                  
 max_pooling2d_6 (MaxPoolin  (None, 3, 3, 32)             0         ['conv2d_12[0][0]']           
 g2D)                                                                                             
                                                                                                  
 conv2d_13 (Conv2D)          (None, 1, 1, 64)             18496     ['max_pooling2d_6[0][0]'

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Classification loss
              metrics=['accuracy'])

In [ ]:
# Assuming each coordinate can take values from 0 to 7
num_classes = 8

# Split the labels_data into four separate arrays
labels1 = labels_data[:, 0]  # First coordinate
labels2 = labels_data[:, 1]  # Second coordinate
labels3 = labels_data[:, 2]  # Third coordinate
labels4 = labels_data[:, 3]  # Fourth coordinate

# One-hot encode each of the label arrays
labels1 = to_categorical(labels1, num_classes=num_classes)
labels2 = to_categorical(labels2, num_classes=num_classes)
labels3 = to_categorical(labels3, num_classes=num_classes)
labels4 = to_categorical(labels4, num_classes=num_classes)

In [ ]:
# Train the model
history = model.fit(training_data, [labels1, labels2, labels3, labels4], epochs=10, validation_split=0.2)

Epoch 1/10
280/280 [==============================] - 5s 10ms/step - loss: 7.8701 - output1_loss: 1.8666 - output2_loss: 1.9114 - output3_loss: 2.0231 - output4_loss: 2.0690 - output1_accuracy: 0.2600 - output2_accuracy: 0.2630 - output3_accuracy: 0.1837 - output4_accuracy: 0.1709 - val_loss: 8.7949 - val_output1_loss: 2.1980 - val_output2_loss: 2.3038 - val_output3_loss: 2.1151 - val_output4_loss: 2.1780 - val_output1_accuracy: 0.0760 - val_output2_accuracy: 0.1349 - val_output3_accuracy: 0.1430 - val_output4_accuracy: 0.0965
Epoch 2/10
280/280 [==============================] - 2s 8ms/step - loss: 7.3414 - output1_loss: 1.6308 - output2_loss: 1.7053 - output3_loss: 1.9744 - output4_loss: 2.0309 - output1_accuracy: 0.3595 - output2_accuracy: 0.3422 - output3_accuracy: 0.2113 - output4_accuracy: 0.1956 - val_loss: 9.1196 - val_output1_loss: 2.2766 - val_output2_loss: 2.4638 - val_output3_loss: 2.1717 - val_output4_loss: 2.2076 - val_output1_accuracy: 0.1488 - val_output2_accuracy: 0.12

In [ ]:
# Save model weights
model.save_weights('cnn_weights2.h5')